In [1]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
frame = pd.read_csv("botswana.tsv", sep="\t", index_col=False) 
frame.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [4]:
for col in frame.columns:
    print col, len(frame[col].value_counts())

ceb 14
age 35
educ 21
religion 4
idlnchld 19
knowmeth 2
usemeth 2
evermarr 2
agefm 35
heduc 21
urban 2
electric 2
radio 2
tv 2
bicycle 2


In [5]:
naframe=frame.dropna()
naframe.shape

(1834, 15)

In [6]:
frame.drop('evermarr', axis=1, inplace=True)

In [7]:
vec = map(np.int,frame.agefm.isnull())
frame['nevermarr']=vec
frame.agefm = frame.agefm.fillna(0)

In [8]:
frame.heduc.loc[(frame.nevermarr==1)]=-1

/home/ivanov/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
frame.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [10]:
frame.heduc.isnull().value_counts()

False    4238
True      123
Name: heduc, dtype: int64

In [11]:
vec = map(np.int,frame.idlnchld.isnull())
frame['idlnchld_noans']=vec
frame.idlnchld = frame.idlnchld.fillna(-1)

vec = map(np.int,frame.heduc.isnull())
frame['heduc_noans']=vec
frame.heduc = frame.heduc.fillna(-2)

vec = map(np.int,frame.usemeth.isnull())
frame['usemeth_noans']=vec
frame.usemeth = frame.usemeth.fillna(-1)

In [12]:
frame.drop(frame[frame.bicycle.isnull()].index,inplace=True)
frame.drop(frame[frame.knowmeth.isnull()].index,inplace=True)
frame.drop(frame[frame.electric.isnull()].index,inplace=True)
frame.drop(frame[frame.radio.isnull()].index,inplace=True)
frame.drop(frame[frame.tv.isnull()].index,inplace=True)

In [13]:
np.product(frame.shape)

78264

In [14]:
frame.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


In [15]:
signs = "age + educ + religion + idlnchld"\
"+ knowmeth + usemeth + agefm + heduc + urban"\
"+ electric + radio + tv + bicycle + nevermarr"\
"+ idlnchld_noans + heduc_noans + usemeth_noans"

In [16]:
opis = 'ceb ~ '+signs

In [17]:
m1 = smf.ols("ceb ~ age + educ + religion + idlnchld"\
"+ knowmeth + usemeth + agefm + heduc + urban"\
"+ electric + radio + tv + bicycle + nevermarr"\
"+ idlnchld_noans + heduc_noans + usemeth_noans", 
             data=frame)
fitted = m1.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Fri, 20 Jan 2017   Prob (F-statistic):               0.00
Time:                        22:40:40   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                 -1

In [18]:
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]

Breusch-Pagan test: p=0.000000


In [19]:
m2 = smf.ols("ceb ~ age + educ + religion + idlnchld"\
"+ knowmeth + usemeth + agefm + heduc + urban"\
"+ electric + radio + tv + bicycle + nevermarr"\
"+ idlnchld_noans + heduc_noans + usemeth_noans", 
             data=frame)
fitted = m2.fit(cov_type='HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Fri, 20 Jan 2017   Prob (F-statistic):               0.00
Time:                        22:40:40   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                 -1

In [20]:
frame.drop(['religion','radio','tv'],1,inplace=True)

In [21]:
m3 = smf.ols("ceb ~ age + educ + idlnchld"\
"+ knowmeth + usemeth + agefm + heduc + urban"\
"+ electric + + bicycle + nevermarr"\
"+ idlnchld_noans + heduc_noans + usemeth_noans", 
             data=frame)
fitted = m3.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Fri, 20 Jan 2017   Prob (F-statistic):               0.00
Time:                        22:40:40   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.

In [22]:
print 'Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1]

Breusch-Pagan test: p=0.000000


In [23]:
m4 = smf.ols("ceb ~ age + educ + idlnchld"\
"+ knowmeth + usemeth + agefm + heduc + urban"\
"+ electric + + bicycle + nevermarr"\
"+ idlnchld_noans + heduc_noans + usemeth_noans", 
             data=frame)
fitted = m4.fit(cov_type='HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Fri, 20 Jan 2017   Prob (F-statistic):               0.00
Time:                        22:40:41   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [24]:
print "F=%f, p=%f, k1=%f" % m2.fit().compare_f_test(m4.fit())

F=0.919236, p=0.467231, k1=5.000000


In [ ]:
frame.drop(['usemeth_noans','usemeth'],1,inplace=True)

In [ ]:
m5 = smf.ols("ceb ~ age + educ + idlnchld"\
"+ knowmeth + agefm + heduc + urban"\
"+ electric + + bicycle + nevermarr"\
"+ idlnchld_noans + heduc_noans", 
             data=frame)
fitted = m5.fit()
print fitted.summary()

In [ ]:
print "F=%f, p=%f, k1=%f" % m3.fit().compare_f_test(m5.fit())